# HW 2 - Naive Bayes in Hadoop MR
__`MIDS w261: Machine Learning at Scale | UC Berkeley School of Information | Fall 2018`__

In the live sessions for week 2 and week 3 you got some practice designing and debugging Hadoop Streaming jobs. In this homework we'll use Hadoop MapReduce to implement your first parallelized machine learning algorithm: Naive Bayes. As you develop your implementation you'll test it on a small dataset that matches the 'Chinese Example' in the _Manning, Raghavan and Shutze_ reading for Week 2. For the main task in this assignment you'll be working with a small subset of the Enron Spam/Ham Corpus. By the end of this assignment you should be able to:
* __... describe__ the Naive Bayes algorithm including both training and inference.
* __... perform__ EDA on a corpus using Hadoop MR.
* __... implement__ parallelized Naive Bayes.
* __... constrast__ partial, unordered and total order sort and their implementations in Hadoop Streaming.
* __... explain__ how smoothing affects the bias and variance of a Multinomial Naive Bayes model.

As always, your work will be graded both on the correctness of your output and on the clarity and design of your code. __Please refer to the `README` for homework submission instructions.__ 

## Notebook Setup
Before starting, run the following cells to confirm your setup.

In [1]:
# imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
# global vars (paths) - ADJUST AS NEEDED
JAR_FILE = "/usr/lib/hadoop-mapreduce/hadoop-streaming.jar"
#HDFS_DIR = "/user/root/HW2/"
HDFS_DIR = "/user/root/HW2"
HOME_DIR = "/media/notebooks/sp21-andreaslemos/Assignments/HW2" # FILL IN HERE eg. /media/notebooks/Assignments/HW2

In [3]:
# global vars (paths) - ADJUST AS NEEDED - Adjusted by Andreas
#JAR_FILE = "/usr/lib/hadoop-mapreduce/hadoop-streaming.jar"
#HDFS_DIR = "/home/ec2-user/sp21-andreaslemos/Assignments/HW2"
#HOME_DIR = "" # FILL IN HERE eg. /media/notebooks/Assignments/HW2

In [4]:
### Added by Andreas
!pwd

/media/notebooks/sp21-andreaslemos/Assignments/HW2


In [5]:
# save path for use in Hadoop jobs (-cmdenv PATH={PATH})
from os import environ
PATH  = environ['PATH']

In [6]:
# data path
ENRON = "data/enronemail_1h.txt"

In [7]:
# make the HDFS directory if it doesn't already exist
!hdfs dfs -ls 
!hdfs dfs -mkdir {HDFS_DIR}

Found 1 items
drwxr-xr-x   - root supergroup          0 2021-01-21 22:08 demo3


# Question 1: Hadoop MapReduce Key Takeaways.  

This assignment will be the only one in which you use Hadoop Streaming to implement a distributed algorithm. The key reason we continue to teach Hadoop streaming is because of the way it forces the programmer to think carefully about what is happening under the hood when you parallelize a calculation. This question will briefly highlight some of the most important concepts that you need to understand about Hadoop Streaming and MapReduce before we move on to Spark next week.   

### Q1 Tasks:

* __a) short response:__ What "programming paradigm" is Hadoop MapReduce based on? What are the main ideas of this programming paradigm and how does MapReduce exemplify these ideas?

* __b) short response:__ What is the Hadoop Shuffle? When does it happen? Why is it potentially costly? Describe one specific thing we can we do to mitigate the cost associated with this stage of our Hadoop Streaming jobs.

* __c) short response:__ In Hadoop Streaming why do the input and output record format of a combiner script have to be the same? [__`HINT`__ _what level of combining does the framework guarantee? what is the relationship between the record format your mapper emits and the format your reducer expects to receive?_]

* __d) short response:__ To what extent can you control the level of parallelization of your Hadoop Streaming jobs? Please be specific.

* __e) short response:__ What change in the kind of computing resources available prompted the creation of parallel computation frameworks like Hadoop? 

### Q1 Student Answers:

> __a)__ Haddop MapReduce is based on the 'functional programming' paradigm. In this paradigm, the output values of a function depend only on the inputs to that function. The key idea being that it is a 'stateless' paradigm, which uses higher-order functions. Programming in thhis paradigm is done in expressions. MapReduce exemplify these ideas in the stateless and higher-order functions such as Map and Reduce.

> __b)__ Hadoop Shuffle is the a synchronization process that happens between the Map and Reduce phases. The idea is that the Reduce pahse does not start until all the Mappers are done. And when they are done, the Shuffling step reorganizes the output from the Mapper so they are sorted by key before they are used as input to the Reducer. It is costly becasue of the sorting and network transfer involved in this step.

NEED EXAMPLE OF WHAT CAN BE DONE TO REDUCE COST!

> __c)__ Type your answer here!  

ANSWER THIS

> __d)__ You can hint the level of parallelization for the Map phase. However, Hadoop will ultimately decide what the most efficient level is based on the physical layout of the data. On the other hand, for the Reduce phase you can precisely specify the number of reduce tasks. 

REVIEW  

> __e)__ Type your answer here!  

COMPUTING IN CLUSTERS???

# Question 2: MapReduce Design Patterns.  

In the last two live sessions and in your readings from Lin & Dyer you encountered a number of techniques for manipulating the logistics of a MapReduce implementation to ensure that the right information is available at the right time and location. In this question we'll review a few of the key techniques you learned.   

### Q2 Tasks:

* __a) short response:__ What are counters (in the context of Hadoop Streaming)? How are they useful? What kinds of counters does Hadoop provide for you? How do you create your own custom counter?

* __b) short response:__ What are composite keys? How are they useful? How are they related to the idea of custom partitioning?

* __c) short response:__ What is the order inversion pattern? What problem does it help solve? How do we implement it? 

### Q2 Student Answers:

> __a)__ Haddop counter provide a means to keep track of progress for a particular MapReduce job. They are useful for tracking progress and stats, for quality control, and troubleshooting. Hadoop provides two types of counters: Built-in counters and User-defined counters. Customer counters can be created in Java, for example, using 'enum' to define it. The name of the 'enum' is the group name, and the enum fields are the counter names.

> __b)__ Composite keys are keys built using the first and second fields so a secondary sort can be achieved. They are useful for problems where a secondary sort needs to be done on data that has already been preocessed a first time. In order to achieve this, a Custom Partitioner function needs to be created in otder to specify how to partition keys passed to the reducers.

> __c)__ The order inversion pattern can be used to control the order of reducer values in the MapReduce framework.Order inversion patterns use less memory and simpler data structures. That is beacuse there is no additioanl soprting and ordering needed in the reducer phase. The order inversion pattern is applied at the data analysis phase, ordering the data before it arrives at the Reduce phase.


# Question 3: Understanding Total Order Sort

The key challenge in distributed computing is to break a problem into a set of sub-problems that can be performed without communicating with each other. Ideally, we should be able to define an arbirtary number of splits and still get the right result, but that is not always possible. Parallelization becomes particularly challenging when we need to make comparisons between records, for example when sorting. Total Order Sort allows us to order large datasets in a way that enables efficient retrieval of results. Before beginning this assignment, make sure you have read and understand the [Total Order Sort Notebook](https://github.com/UCB-w261/main/tree/master/HelpfulResources/TotalSortGuide/_total-sort-guide-spark2.01-JAN27-2017.ipynb). You can skip the first two MRJob sections, but the rest of section III and all of section IV are **very** important (and apply to Hadoop Streaming) so make sure to read them closely. Feel free to read the Spark sections as well but you won't be responsible for that material until later in the course. To verify your understanding, answer the following questions.

### Q3 Tasks:

* __a) short response:__ What is the difference between a partial sort, an unordered total sort, and a total order sort? From the programmer's perspective, what does total order sort allow us to do that we can't with unordered total? Why is this important with large datasets?

* __b) short response:__ Which phase of a MapReduce job is leveraged to implement Total Order Sort? Which default behaviors must be changed. Why must they be changed?

* __c) short response:__ Describe in words how to configure a Hadoop Streaming job for the custom sorting and partitioning that is required for Total Order Sort.  

* __d) short response:__ Explain why we need to use an inverse hash code function.

* __e) short response:__ Where does this function need to be located so that a Total Order Sort can be performed?

### Q3 Student Answers:

> __a)__ In a partial sort, keys are assigned to partitions without any order, and are then sorted within each partition. In the case of an unordered sort, keys are assigned to partitions based on their numeric values. For example, one partition can hold all keys between 0 and 99, another can have all keys between 100 and 199, and so on. The keys within a partition are sorted, but there is no order between the partitions. Finally, the total order sort ensure that, in addition to the keys being assigned to partitions based on their numeric value and the keys being sorted in the partitions, the partitions themselves are assigned in order so that the key groups are in the correct sequence. 

# From the programmer's perspective, what does total order sort allow us to do that we can't with unordered total? Why is this important with large datasets?

> __b)__ Hadoop's architecture only guarantees partial sort in it's shuffling and sort phase. In order to achieve Total Order Sort, three default behaviors must be changed. First, a partition file or function needs to exist inside the Mapper. This is where a partition key is generated to instruct Haddop how to partition records. Second, we need to drop the internal partition key inside the Reducer. This is where the key used for partitioning is drop and the reducer recovers the orignal (key, value) pairs. Finally, there is a post-processing step to order the partitions. While the partitions (i.e., files names) are ordered, the partitions keys may not be.

> __c)__ Type your answer here!  

# Need to understand what they are asking

> __d)__ The inverse hash code function is needed in order to preserve partition key ordering. Th input to the inverse hash function is the desired partition index and total number of partitions. THe output of this fucntion is a particular partition key.

> __e)__ This inverse hash function needs to be placed in the Mapper for the Total Order Sort to be performed.

# About the Data
For the main task in this portion of the homework you will train a classifier to determine whether an email represents spam or not. You will train your Naive Bayes model on a 100 record subset of the Enron Spam/Ham corpus available in the HW2 data directory (__`HW2/data/enronemail_1h.txt`__).

__Source:__   
The original data included about 93,000 emails which were made public after the company's collapse. There have been a number raw and preprocessed versions of this corpus (including those available [here](http://www.aueb.gr/users/ion/data/enron-spam/index.html) and [here](http://www.aueb.gr/users/ion/publications.html)). The subset we will use is limited to emails from 6 Enron employees and a number of spam sources. It is part of [this data set](http://www.aueb.gr/users/ion/data/enron-spam/) which was created by researchers working on personlized Bayesian spam filters. Their original publication is [available here](http://www.aueb.gr/users/ion/docs/ceas2006_paper.pdf). __`IMPORTANT!`__ _For this homework please limit your analysis to the 100 email subset which we provide. No need to download or run your analysis on any of the original datasets, those links are merely provided as context._

__Preprocessing:__  
For their work, Metsis et al. (the authors) appeared to have pre-processed the data, not only collapsing all text to lower-case, but additionally separating "words" by spaces, where "words" unfortunately include punctuation. As a concrete example, the sentence:  
>  `Hey Jon, I hope you don't get lost out there this weekend!`  

... would have been reduced by Metsis et al. to the form:  
> `hey jon , i hope you don ' t get lost out there this weekend !` 

... so we have reverted the data back toward its original state, removing spaces so that our sample sentence would now look like:
> `hey jon, i hope you don't get lost out there this weekend!`  

Thus we have at least preserved contractions and other higher-order lexical forms. However, one must be aware that this reversion is not complete, and that some object (specifically web sites) will be ill-formatted, and that all text is still lower-cased.


__Format:__   
All messages are collated to a tab-delimited format:  

>    `ID \t SPAM \t SUBJECT \t CONTENT \n`  

where:  
>    `ID = string; unique message identifier`  
    `SPAM = binary; with 1 indicating a spam message`  
    `SUBJECT = string; title of the message`  
    `CONTENT = string; content of the message`   
    
Note that either of `SUBJECT` or `CONTENT` may be "NA", and that all tab (\t) and newline (\n) characters have been removed from both of the `SUBJECT` and `CONTENT` columns.  

In [8]:
!pwd

/media/notebooks/sp21-andreaslemos/Assignments/HW2


In [9]:
# take a look at the first 100 characters of the first 5 records (RUN THIS CELL AS IS)
# !head -n 5 /media/notebooks/Assignments/HW2/{ENRON} | cut -c-100
!head -n 5 /media/notebooks/sp21-andreaslemos/Assignments/HW2/{ENRON} | cut -c-100 # Adjusted by Andreas

0001.1999-12-10.farmer	0	 christmas tree farm pictures	NA
0001.1999-12-10.kaminski	0	 re: rankings	 thank you.
0001.2000-01-17.beck	0	 leadership development pilot	" sally:  what timing, ask and you shall receiv
0001.2000-06-06.lokay	0	" key dates and impact of upcoming sap implementation over the next few week
0001.2001-02-07.kitchen	0	 key hr issues going forward	 a) year end reviews-report needs generating 


In [10]:
# see how many messages/lines are in the file 
#(this number may be off by 1 if the last line doesn't end with a newline)
# !wc -l /media/notebooks/Assignments/HW2/{ENRON}
!wc -l /media/notebooks/sp21-andreaslemos/Assignments/HW2/{ENRON} # Adjusted by Andreas

100 /media/notebooks/sp21-andreaslemos/Assignments/HW2/data/enronemail_1h.txt


In [11]:
# make the HDFS directory if it doesn't already exist
!hdfs dfs -mkdir {HDFS_DIR}

mkdir: `/user/root/HW2': File exists


In [12]:
# load the data into HDFS (RUN THIS CELL AS IS)
# !hdfs dfs -copyFromLocal /media/notebooks/Assignments/HW2/{ENRON} {HDFS_DIR}/enron.txt
!hdfs dfs -copyFromLocal /media/notebooks/sp21-andreaslemos/Assignments/HW2/{ENRON} {HDFS_DIR}/enron.txt # Adjusted by Andreas

In [13]:
!hdfs dfs -ls {HDFS_DIR}

Found 1 items
-rw-r--r--   1 root supergroup     204559 2021-01-21 22:18 /user/root/HW2/enron.txt


# Question 4:  Enron Ham/Spam EDA.
Before building our classifier, lets get aquainted with our data. In particular, we're interested in which words occur more in spam emails than in legitimate ("ham") emails. In this question you'll implement two Hadoop MapReduce jobs to count and sort word occurrences by document class. You'll also learn about two new Hadoop streaming parameters that will allow you to control how the records output from your mappers are partitioned for reducing on separate nodes. 

__`IMPORTANT NOTE:`__ For this question and all subsequent items, you should include both the subject and the body of the email in your analysis (i.e. concatetate them to get the 'text' of the document).

### Q4 Tasks:
* __a) code:__ Complete the missing components of the code in __`EnronEDA/mapper.py`__ and __`EnronEDA/reducer.py`__ to create a Hadoop  MapReduce job that counts how many times each word in the corpus occurs in an email for each class. Pay close attention to the data format specified in the docstrings of these scripts _-- there are a number of ways to accomplish this task, we've chosen this format to help illustrate a technique in `part e`_. Run the provided unit tests to confirm that your code works as expected, then run the provided Hadoop Streaming command to apply your analysis to the Enron data.


* __b) code + short response:__ How many times does the word "__assistance__" occur in each class? (`HINT:` Use a `grep` command to read from the results file you generated in '`a`' and then report the answer in the space provided.)


* __c) short response:__ Would it have been possible to add some sorting parameters to the Hadoop streaming command that would cause our `part a` results to be sorted by count? Explain why or why not. (`HINT:` This question demands an understanding of the sequence of the phases of MapReduce.)


* __d) code + short response:__ Write a second Hadoop MapReduce job to sort the output of `part a` first by class and then by count. Run your job and save the results to a local file. Then describe in words how you would go about printing the top 10 words in each class given this sorted output. (`HINT 1:` _remember that you can simply pass the `part a` output directory to the input field of this job; `HINT 2:` since this task is just reodering the records from `part a` we don't need to write a mapper or reducer, just use `/bin/cat` for both_)


* __e) code:__ A more efficient alternative to '`grep`-ing' for the top 10 words in each class would be to use the Hadoop framework to separate records from each class into its own partition so that we can just read the top lines in each. Rewrite your job from ` part d` to specify 2 reduce tasks and to tell Hadoop to partition based on the second field (which indicates spam/ham in our data). Your code should maintain the secondary sort -- that is each partition should list words from most to least frequent.

### Q4 Student Answers:
> __b)__ Type your answer here!  

> __c)__ Type your answer here!  

> __d)__ Type your answer here!  

In [14]:
# part a - do your work in the provided scripts then RUN THIS CELL AS IS
!chmod a+x EnronEDA/mapper.py
!chmod a+x EnronEDA/reducer.py

In [15]:
# part a - unit test EnronEDA/mapper.py (RUN THIS CELL AS IS)
!echo -e "d1	1	title	body\nd2	0	title	body" | EnronEDA/mapper.py

title	1	1
body	1	1
title	0	1
body	0	1


In [16]:
# part a - unit test EnronEDA/reducer.py (RUN THIS CELL AS IS)
!echo -e "one	1	1\none	0	1\none	0	1\ntwo	0	1" | EnronEDA/reducer.py

one	0	2
one	1	1
two	0	1


In [17]:
# part a - clear output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-output

rm: `/user/root/HW2/eda-output': No such file or directory


In [18]:
# part a - Hadoop streaming job (RUN THIS CELL AS IS)
!hadoop jar {JAR_FILE} \
  -files EnronEDA/reducer.py,EnronEDA/mapper.py \
  -mapper mapper.py \
  -reducer reducer.py \
  -input {HDFS_DIR}/enron.txt \
  -output {HDFS_DIR}/eda-output \
  -numReduceTasks 1 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob5332596431018807321.jar tmpDir=null
21/01/21 22:18:58 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/21 22:18:58 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/21 22:18:59 INFO mapred.FileInputFormat: Total input paths to process : 1
21/01/21 22:18:59 INFO mapreduce.JobSubmitter: number of splits:2
21/01/21 22:18:59 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1611186257570_0003
21/01/21 22:18:59 INFO impl.YarnClientImpl: Submitted application application_1611186257570_0003
21/01/21 22:18:59 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1611186257570_0003/
21/01/21 22:18:59 INFO mapreduce.Job: Running job: job_1611186257570_0003
21/01/21 22:19:05 INFO mapreduce.Job: Job job_1611186257570_0003 running in uber mode : false
21/01/21 22:19:05 INFO mapreduce.Job:  map 0% reduce 0%
21/01

In [19]:
# part a - retrieve results from HDFS & copy them into a local file (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/eda-output/part-0000* > EnronEDA/results.txt

In [20]:
# part b - write your grep command here
!grep 'assistance' EnronEDA/results.txt

assistance	0	2
assistance	1	8


In [21]:
# part d - clear the output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-sort-output

rm: `/user/root/HW2/eda-sort-output': No such file or directory


In [22]:
### part d - write your Hadoop streaming job here
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=3 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k2n -k3nr -k1" \
  -mapper /bin/cat \
  -reducer /bin/cat \
  -input {HDFS_DIR}/eda-output \
  -output {HDFS_DIR}/eda-sort-output \
  -numReduceTasks 1 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob7862240239962594421.jar tmpDir=null
21/01/21 22:19:45 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/21 22:19:45 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/21 22:19:45 INFO mapred.FileInputFormat: Total input paths to process : 1
21/01/21 22:19:45 INFO mapreduce.JobSubmitter: number of splits:2
21/01/21 22:19:45 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1611186257570_0004
21/01/21 22:19:46 INFO impl.YarnClientImpl: Submitted application application_1611186257570_0004
21/01/21 22:19:46 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1611186257570_0004/
21/01/21 22:19:46 INFO mapreduce.Job: Running job: job_1611186257570_0004
21/01/21 22:19:52 INFO mapreduce.Job: Job job_1611186257570_0004 running in uber mode : false
21/01/21 22:19:52 INFO mapreduce.Job:  map 0% reduce 0%
21/01

In [23]:
### Added by Andreas - but I think it's needed
# part d - retrieve results from HDFS & copy them into a local file (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/eda-sort-output/part-0000* > EnronEDA/results_sort.txt

In [24]:
# part e - clear the output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-sort-output

Deleted /user/root/HW2/eda-sort-output


In [25]:
# part e - write your Hadoop streaming job here
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=3 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k2n -k3nr -k1" \
  -D mapreduce.partition.keypartitioner.options="-k2,2n" \
  -mapper /bin/cat \
  -reducer /bin/cat \
  -input {HDFS_DIR}/eda-output \
  -output {HDFS_DIR}/eda-sort-output \
  -numReduceTasks 2 \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob1025769895919047718.jar tmpDir=null
21/01/21 22:21:15 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/21 22:21:15 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/21 22:21:16 INFO mapred.FileInputFormat: Total input paths to process : 1
21/01/21 22:21:16 INFO mapreduce.JobSubmitter: number of splits:2
21/01/21 22:21:16 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1611186257570_0005
21/01/21 22:21:16 INFO impl.YarnClientImpl: Submitted application application_1611186257570_0005
21/01/21 22:21:16 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1611186257570_0005/
21/01/21 22:21:16 INFO mapreduce.Job: Running job: job_1611186257570_0005
21/01/21 22:21:22 INFO mapreduce.Job: Job job_1611186257570_0005 running in uber mode : false
21/01/21 22:21:22 INFO mapreduce.Job:  map 0% reduce 0%
21/01

In [26]:
# part e - view the top 10 records from each partition (RUN THIS CELL AS IS)
for idx in range(2):
    print(f"\n===== part-0000{idx}=====\n")
    !hdfs dfs -cat {HDFS_DIR}/eda-sort-output/part-0000{idx} | head


===== part-00000=====

the	0	549	
to	0	398	
ect	0	382	
and	0	278	
of	0	230	
hou	0	206	
a	0	196	
in	0	182	
for	0	170	
on	0	135	
cat: Unable to write to output stream.

===== part-00001=====

the	1	698	
to	1	566	
and	1	392	
your	1	357	
a	1	347	
you	1	345	
of	1	336	
in	1	236	
for	1	204	
com	1	153	
cat: Unable to write to output stream.


__Expected output:__
<table>
<th>part-00000:</th>
<th>part-00001:</th>
<tr><td><pre>
the	0	549	
to	0	398	
ect	0	382	
and	0	278	
of	0	230	
hou	0	206	
a	0	196	
in	0	182	
for	0	170	
on	0	135
</pre></td>
<td><pre>
the	1	698	
to	1	566	
and	1	392	
your	1	357	
a	1	347	
you	1	345	
of	1	336	
in	1	236	
for	1	204	
com	1	153
</pre></td></tr>
</table>

# Question 5: Counters and Combiners.
Tuning the number of mappers & reducers is helpful to optimize very large distributed computations. Doing so successfully requires a thorough understanding of the data size at each stage of the job. As you learned in the week3 live session, counters are an invaluable resource for understanding this kind of detail. In this question, we will take the EDA performed in Question 4 as an opportunity to illustrate some related concepts.

### Q5 Tasks:
* __a) short response:__ Read the Hadoop output from your job in Question 4a to report how many records are emitted by the mappers and how many records are received be the reducers. In the context of word counting what does this number represent practically?

* __b) code:__ Note that we wrote the reducer in question 4a such that the input and output record format is identical. This makes it easy to use the same reducer script as a combiner. In the space provided below, write the Hadoop Streaming command to re-run your job from question 4a with this combining added.

* __c) short response__: Report the number of records emitted by your mappers in part b and the number of records received by your reducers. Compare your results here to what you saw in part a. Explain.

* __d) short response__: Describe a scenario where using a combiner would _NOT_ improve the efficiency of the shuffle stage. Explain. [__`BONUS:`__ how does increasing the number of mappers affect the usefulness of a combiner?]

### Q5 Student Answers:
> __a)__ Based on the output from 4a above (relevant lines displyed below), the mappers emitted 31,490 records as their output (Map output records=31490). Those same 31,490 are received by the Reducers (Reduce input records=31490). From a word counting perspective, this represents the total number of words in the text.

- From 4a
Map-Reduce Framework
		Map input records=100
		Map output records=31490
        ...
		Reduce input records=31490
		Reduce output records=6060


> __c)__ The number of records emitted by the Mappers remains the same at 31,490, as expected. However, with the Combiner included, the number of records recieved by the Reducer now has decresed signficantly, from 31,490 top 7,648. This is due to the fact that the Combiner is doing a local aggregation of records. This means that fewer records (i.e., lines) need to be passed to the Reducer.

- From 5b
Map-Reduce Framework
		Map input records=100
		Map output records=31490
        ...
		Reduce input records=7648
        Reduce output records=6061

> __d)__ If the numbers of mappers is close to the number of words, then any local aggregation would not significanlty improve the efficiency of the shuffle stage. that is beacuse there would be very little local aggregation available. Therefore, increasing the number of mappers DECREASES the usefulness of a combiner, since less local aggregation is possible. 

In [30]:
# part b - clear output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-output

Deleted /user/root/HW2/eda-output


In [31]:
# part b - write your Hadoop streaming job here
!hadoop jar {JAR_FILE} \
  -files EnronEDA/reducer.py,EnronEDA/mapper.py,EnronEDA/combiner.py \
  -mapper mapper.py \
  -combiner combiner.py \
  -reducer reducer.py \
  -input {HDFS_DIR}/enron.txt \
  -output {HDFS_DIR}/eda-output \
  -numReduceTasks 1 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob480988166729164389.jar tmpDir=null
21/01/21 22:50:57 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/21 22:50:57 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/21 22:50:58 INFO mapred.FileInputFormat: Total input paths to process : 1
21/01/21 22:50:58 INFO mapreduce.JobSubmitter: number of splits:2
21/01/21 22:50:58 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1611186257570_0007
21/01/21 22:50:58 INFO impl.YarnClientImpl: Submitted application application_1611186257570_0007
21/01/21 22:50:58 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1611186257570_0007/
21/01/21 22:50:58 INFO mapreduce.Job: Running job: job_1611186257570_0007
21/01/21 22:51:04 INFO mapreduce.Job: Job job_1611186257570_0007 running in uber mode : false
21/01/21 22:51:04 INFO mapreduce.Job:  map 0% reduce 0%
21/01/

# Question 6: Document Classification Task Overview.
The week 2 assigned reading from Chapter 13 of _Introduction to Information Retrieval_ by Manning, Raghavan and Schutze provides a thorough introduction to the document classification task and the math behind Naive Bayes. In this question we'll use the example from Table 13.1 (reproduced below) to 'train' an unsmoothed Multinomial Naive Bayes model and classify a test document by hand.

<table>
<th>DocID</th>
<th>Class</th>
<th>Subject</th>
<th>Body</th>
<tr><td>Doc1</td><td>1</td><td></td><td>Chinese Beijing Chinese</td></tr>
<tr><td>Doc2</td><td>1</td><td></td><td>Chinese Chinese Shanghai</td></tr>
<tr><td>Doc3</td><td>1</td><td></td><td>Chinese Macao</td></tr>
<tr><td>Doc4</td><td>0</td><td></td><td>Tokyo Japan Chinese</td></tr>
</table>

### Q6 Tasks:
* __a) short response:__ Equation 13.3 in Manning, Raghavan and Shutze shows how a Multinomial Naive Bayes model classifies a document. It predicts the class, $c$, for which the estimated conditional probability of the class given the document's contents,  $\hat{P}(c|d)$, is greatest. In this equation what two pieces of information are required to calculate  $\hat{P}(c|d)$? Your answer should include both mathematical notatation and verbal explanation.


* __b) short response:__ The Enron data includes two classes of documents: `spam` and `ham` (they're actually labeled `1` and `0`). In plain English, explain what  $\hat{P}(c)$ and   $\hat{P}(t_{k} | c)$ mean in the context of this data. How will we would estimate these values from a training corpus? How many passes over the data would we need to make to retrieve this information for all classes and all words?


* __c) hand calculations:__ Above we've reproduced the document classification example from the textbook (we added an empty subject field to mimic the Enron data format). Remember that the classes in this "Chinese Example" are `1` (about China) and `0` (not about China). Calculate the class priors and the conditional probabilities for an __unsmoothed__ Multinomial Naive Bayes model trained on this data. Show the calculations that lead to your result using markdown and $\LaTeX$ in the space provided or by embedding an image of your hand written work. [`NOTE:` _Your results should NOT match those in the text -- they are training a model with +1 smoothing you are training a model without smoothing_]


* __d) hand calculations:__ Use the model you trained to classify the following test document: `Chinese Chinese Chinese Tokyo Japan`. Show the calculations that lead to your result using markdown and   $\LaTeX$ in the space provided or by embedding an image of your hand written work.


* __e) short response:__ Compare the classification you get from this unsmoothed model in `d`/`e` to the results in the textbook's "Example 1" which reflects a model with Laplace plus 1 smoothing. How does smoothing affect our inference?

### Q6 Student Answers:
> __a)__ In order to calculate $\hat{P}(c|d)$, we need two pieces of information. First, we need $\hat{P}(t_k|c)$. This is the conditional probability of term $t_k$ occurring in a document of class $c$. Secondly, we need $\hat{P}(c)$, which is the prior probability of a document occurring in class $c$.

> __b)__ Using the Enron data as an example, $\hat{P}(c)$ would mean the probability of a particular email occuring in a given class (spam or ham). And $\hat{P}(t_k|c)$ is the probability of a given term occuring in a document of a particular class (spma or ham). 

>To estimate $\hat{P}(c)$, one can use the the relatice frequencies of the classes. In other words:

$$\hat{P}(c)=\frac{N_c}{N}$$

>where $N_c$ is the number of documents in class $c$, and $N$ is the total number of documents.

>To estimate $\hat{P}(t_k|c)$, we use the relative frequency of term $t$ in documents belonging to class $c$. This is given by the formula:

$$\hat{P}(t_k|c)=\frac{T_{ct}}{\sum_{t'\in{V}}T_{ct'}'}$$

> __c)__ The prior probablities are straight forward to calculate, given that our example has a total number of documents of 4 ($N=4$), and that the number of documents of class "$1$" is 3 ($N_c=3$). Therefore:

$$\hat{P}(c)=\frac{N_c}{N}=\frac{3}{4}$$
$$\hat{P}(\bar{c})=\frac{1}{4}$$

>The conditional probabilities are calcualted by counting the number of time the terms appear in each class divided by the number of terms in that class. For example, for the first three documents (all of class $1$), the term 'Chinese' appears 5 times, and the total count of terms is 8. This gives us:

$$\hat{P}(Chinese|c)=\frac{5}{8}$$
$$\hat{P}(Beijing|c)=\hat{P}(Shanghai|c)=\hat{P}(Macao|c)=\frac{1}{8}$$
$$\hat{P}(Tokyo|c)=\hat{P}(Japan|c)=0$$

$$\hat{P}(Tokyo|\bar{c})=\hat{P}(Japan|\bar{c})=\hat{P}(Chinese|\bar{c})=\frac{1}{3}$$


> __d)__ Consider the document 'Chinese Chinese Chinese Tokyo Japan' to be $d_5$. We then have that:

$$\hat{P}(c|d_5) \propto \frac{3}{4} \cdot \frac{5}{8} \cdot \frac{5}{8} \cdot \frac{5}{8} \cdot 0 \cdot 0=0$$

$$\hat{P}(\bar{c}|d_5) \propto \frac{1}{4} \cdot \frac{1}{3} \cdot \frac{1}{3} \cdot \frac{1}{3} \cdot \frac{1}{3} \cdot \frac{1}{3}=0.00102$$

>Based on the above, the document 'Chinese Chinese Chinese Tokyo Japan' is classified as a class $0$.


> __e)__ The smoothed classifier, from the book, works better and classifies the document more accurately beacuse the unsmoothed classifier is unable to handle cases where the term does not appear in any of the documents for a given class. The conditional probability for that term then becomes $0$ in the unsmoothed classifier, bringing the overall probability to zero, incorreclty. The smoothed approach corrects this issue by introducing a $+1$ term to the equation. 


In [ ]:
# part d/e - if you didn't write out your calcuations above, embed a picture of them here:
from IPython.display import Image
Image(filename="path-to-hand-calulations-image.png")

# Question 7: Naive Bayes Inference.
In the next two questions you'll write code to parallelize the Naive Bayes calculations that you performed above. We'll do this in two phases: one MapReduce job to perform training and a second MapReduce to perform inference. While in practice we'd need to train a model before we can use it to classify documents, for learning purposes we're going to develop our code in the opposite order. By first focusing on the pieces of information/format we need to perform the classification (inference) task you should find it easier to develop a solid implementation for training phase when you get to question 8 below. In both of these questions we'll continue to use the Chinese example corpus from the textbook to help us test our MapReduce code as we develop it. Below we've reproduced the corpus, test set and model in text format that matches the Enron data.

### Q7 Tasks:
* __a) short response:__ run the provided cells to create the example files and load them in to HDFS. Then take a closer look at __`NBmodel.txt`__. This text file represents a Naive Bayes model trained (with Laplace +1 smoothing) on the example corpus. What are the 'keys' and 'values' in this file? Which record means something slightly different than the rest? The value field of each record includes two numbers which will be helpful for debugging but which we don't actually need to perform inference -- what are they? [`HINT`: _This file represents the model from Example 13.1 in the textbook, if you're having trouble getting oriented try comparing our file to the numbers in that example._]


* __b) short response:__ When performing Naive Bayes in practice instead of multiplying the probabilities (as in equation 13.3) we add their logs (as in equation 13.4). Why do we choose to work with log probabilities? If we had an unsmoothed model, what potential error could arise from this transformation?


* __c) short response:__ Documents 6 and 8 in the test set include a word that did not appear in the training corpus (and as a result does not appear in the model). What should we do at inference time when we need a class conditional probability for this word?


* __d) short response:__ The goal of our MapReduce job is to stream over the test set and classify each document by peforming the calculation from equation 13.4. To do this we'll load the model file (which contains the probabilities for equation 13.4) into memory on the nodes where we do our mapping. This is called an in-memory join. Does loading a model 'state' like this depart from the functional programming principles? Explain why or why not. From a scability perspective when would this kind of memory use be justified? when would it be unwise?


* __e) code:__ Complete the code in __`NaiveBayes/classify_mapper.py`__. Read the docstring carefully to understand how this script should work and the format it should return. Run the provided unit tests to confirm that your script works as expected then write a Hadoop streaming job to classify the Chinese example test set. [`HINT 1:` _you shouldn't need a reducer for this one._ `HINT 2:` _Don't forget to add the model file to the_ `-files` _parameter in your Hadoop streaming job so that it gets shipped to the mapper nodes where it will be accessed by your script._]


* __f) short response:__ In our test example and in the Enron data set we have fairly short documents. Since these fit fine in memory on a mapper node we didn't need a reducer and could just do all of our calculations in the mapper. However with much longer documents (eg. books) we might want a higher level of parallelization -- for example we might want to process parts of a document on different nodes. In this hypothetical scenario how would our algorithm design change? What could the mappers still do? What key-value structure would they emit? What would the reducers have to do as a last step?

# Answer part d

### Q7 Student Answers:
> __a)__ Tthe keys in NBModel.txt are the particular terms, such as 'chinese' and 'tokyo'. The values are the sequence of four numerical elements that follow each key, such as '0.0,1.0,0.111111111111,0.142857142857'.
>The ClassPriors record means something different than the rest. While the others are associated with specific terms, the ClassPriors records has information about the ClassPriors for each class.
> The first two numbers in the value field are not directly used in the inference. They are the counts that the term appear in class '0' and class '1'. The third and fourth numbers are the actual conditional probabilities that are used in the inference.

> __b)__  Using log probabilities allows us to use a Sum of conditional probabilities instead of a Multiplication of them. Such multiplication could result in a floating point underflow. If we used the unsmoothed model, we could get to a situation where we are trying to calculate the log of zero, which is undefined.

> __c)__ Under the smoothed model, this type of case is handled by the $+1$ term that is included in the numerator. Since it appears 0 times in training data, the numerator would just be '$0+1$' and the computation will still be possible.

> __d)__ Type your answer here!

> __e)__ Complete the coding portion of this question before answering 'f'.

> __f)__ Type your answer here!


Run these cells to create the example corpus and model.

In [105]:
%%writefile NaiveBayes/chineseTrain.txt
D1	1		Chinese Beijing Chinese
D2	1		Chinese Chinese Shanghai
D3	1		Chinese Macao
D4	0		Tokyo Japan Chinese

Overwriting NaiveBayes/chineseTrain.txt


In [106]:
%%writefile NaiveBayes/chineseTest.txt
D5	1		Chinese Chinese Chinese Tokyo Japan
D6	1		Beijing Shanghai Trade
D7	0		Japan Macao Tokyo
D8	0		Tokyo Japan Trade

Overwriting NaiveBayes/chineseTest.txt


In [107]:
%%writefile NBmodel.txt
beijing	0.0,1.0,0.111111111111,0.142857142857
chinese	1.0,5.0,0.222222222222,0.428571428571
tokyo	1.0,0.0,0.222222222222,0.0714285714286
shanghai	0.0,1.0,0.111111111111,0.142857142857
ClassPriors	1.0,3.0,0.25,0.75
japan	1.0,0.0,0.222222222222,0.0714285714286
macao	0.0,1.0,0.111111111111,0.142857142857

Overwriting NBmodel.txt


In [223]:
# load the data files into HDFS
!hdfs dfs -copyFromLocal NaiveBayes/chineseTrain.txt {HDFS_DIR}
!hdfs dfs -copyFromLocal NaiveBayes/chineseTest.txt {HDFS_DIR}

copyFromLocal: `/user/root/HW2/chineseTrain.txt': File exists
copyFromLocal: `/user/root/HW2/chineseTest.txt': File exists


Your work for `part e` starts here:

In [110]:
# part e - do your work in NaiveBayes/classify_mapper.py first, then run this cell.
!chmod a+x NaiveBayes/classify_mapper.py

# Just using 'else: pass' if word not in model. Check this

In [111]:
# part e - unit test NaiveBayes/classify_mapper.py (RUN THIS CELL AS IS)
!cat NaiveBayes/chineseTest.txt | NaiveBayes/classify_mapper.py | column -t

d5  1  -8.90668134500626   -8.10769031284611   1
d6  1  -5.780743515794329  -4.179502370564408  1
d7  0  -6.591673732011658  -7.511706880737811  0
d8  0  -4.394449154674438  -5.565796731681498  0


In [186]:
# part e - clear the output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/chinese-output

Deleted /user/root/HW2/chinese-output


In [187]:
# part e - write your Hadooop streaming job here
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/classify_mapper.py,NBmodel.txt \
  -mapper classify_mapper.py \
  -input {HDFS_DIR}/chineseTest.txt \
  -output {HDFS_DIR}/chinese-output \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob4056177011881755139.jar tmpDir=null
21/01/24 01:14:43 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/24 01:14:43 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/24 01:14:43 INFO mapred.FileInputFormat: Total input paths to process : 1
21/01/24 01:14:43 INFO mapreduce.JobSubmitter: number of splits:2
21/01/24 01:14:43 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1611186257570_0014
21/01/24 01:14:44 INFO impl.YarnClientImpl: Submitted application application_1611186257570_0014
21/01/24 01:14:44 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1611186257570_0014/
21/01/24 01:14:44 INFO mapreduce.Job: Running job: job_1611186257570_0014
21/01/24 01:14:50 INFO mapreduce.Job: Job job_1611186257570_0014 running in uber mode : false
21/01/24 01:14:50 INFO mapreduce.Job:  map 0% reduce 0%
21/01

In [188]:
# part e - retrieve test set results from HDFS (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/chinese-output/part-000* > NaiveBayes/chineseResults.txt

In [189]:
# part e - take a look (RUN THIS CELL AS IS)
!cat NaiveBayes/chineseResults.txt | column -t

d5  1  -8.90668134500626   -8.10769031284611   1
d6  1  -5.780743515794329  -4.179502370564408  1
d7  0  -6.591673732011658  -7.511706880737811  0
d8  0  -4.394449154674438  -5.565796731681498  0


<table>
<th> Expected output for the test set:</th>
<tr align=Left><td><pre>
d5	1	-8.90668134	-8.10769031	1
d6	1	-5.78074351	-4.17950237	1
d7	0	-6.59167373	-7.51170688	0
d8	0	-4.39444915	-5.56579673	0
</pre></td><tr>
</table>

# Question 8: Naive Bayes Training.
In Question 7 we used a model that we had trained by hand. Next we'll develop the code to do that same training in parallel, making it suitable for use with larger corpora (like the Enron emails). The end result of the MapReduce job you write in this question should be a model text file that looks just like the example (`NBmodel.txt`) that we created by hand above.

To refresh your memory about the training process take a look at  `6a` and `6b` where you described the pieces of information you'll need to collect in order to encode a Multinomial Naive Bayes model. We now want to retrieve those pieces of information while streaming over a corpus. The bulk of the task will be very similar to the word counting excercises you've already done but you may want to consider a slightly different key-value record structure to efficiently tally counts for each class. 

The most challenging (interesting?) design question will be how to retrieve the totals (# of documents and # of words in documents for each class). Of course, counting these numbers is easy. The hard part is the timing: you'll need to make sure you have the counts totalled up _before_ you start estimating the class conditional probabilities for each word. It would be best (i.e. most scalable) if we could find a way to do this tallying without storing the whole vocabulary in memory... Use an appropriate MapReduce design pattern to implement this efficiently! 


### Q8 Tasks:
* __a) make a plan:__  Fill in the docstrings for __`NaiveBayes/train_mapper.py`__ and __`NaiveBayes/train_reducer.py`__ to appropriately reflect the format that each script will input/output. [`HINT:` _the input files_ (`enronemail_1h.txt` & `chineseTrain.txt`) _have a prespecified format and your output file should match_ `NBmodel.txt` _so you really only have to decide on an internal format for Hadoop_].


* __b) implement it:__ Complete the code in __`NaiveBayes/train_mapper.py`__ and __`NaiveBayes/train_reducer.py`__ so that together they train a Multinomial Naive Bayes model __with no smoothing__. Make sure your end result is formatted correctly (see note above). Test your scripts independently and together (using `chineseTrain.txt` or test input of your own devising). When you are satisfied with your Python code design and run a Hadoop streaming command to run your job in parallel on the __chineseTrain.txt__. Confirm that your trained model matches your hand calculations from Question 5.


* __c) short response:__ We saw in Question 6 that adding Laplace +1 smoothing makes our classifications less sensitve to rare words. However implementing this technique requires access to one additional piece of information that we had not previously used in our Naive Bayes training. What is that extra piece of information? [`HINT:` see equation 13.7 in Manning, Raghavan and Schutze].


* __d) short response:__ There are a couple of approaches that we could take to handle the extra piece of information you identified in `c`: 1) if we knew this extra information beforehand, we could provide it to our reducer as a configurable parameter for the vocab size dynamically (_where would we get it in the first place?_). Or 2) we could compute it in the reducer without storing any bulky information in memory but then we'd need some postprocessing or a second MapReduce job to complete the calculation (_why?_). Breifly explain what is non-ideal about each of these options. 


* __e) code + short response:__ Choose one of the 2 options above. State your choice & reasoning in the space below then use that strategy to complete the code in __`NaiveBayes/train_reducer_smooth.py`__. Test this alternate reducer then write and run a Hadoop streaming job to train an MNB model with smoothing on the Chinese example. Your results should match the model that we provided for you above (and the calculations in the textbook example). __IMPORTANT NOTE:__ For full credit on this question, your code must work with multiple reducers. 

    - [`HINT:` You will need to implement custom partitioning - [Total Order Sort Notebook](https://github.com/UCB-w261/main/tree/master/HelpfulResources/TotalSortGuide/_total-sort-guide-spark2.01-JAN27-2017.ipynb)] 

    - [`HINT:` Don't start from scratch with this one -- you can just copy over your reducer code from part `b` and make the needed modifications]. 



__IMPORTANT NOTE:__ For full credit on this question, your code must work with multiple reducers. [`HINT:`_You will need to implement custom partitioning - [Total Order Sort Notebook](https://github.com/UCB-w261/main/tree/master/HelpfulResources/TotalSortGuide/_total-sort-guide-spark2.01-JAN27-2017.ipynb)]


### Q8 Student Answers:
> __ c)__ where B = |V| is the number of terms in the vocabulary

> __ d)__ Type your answer here!

> __ e)__ Type your answer here!


In [114]:
# part a - do your work in train_mapper.py and train_reducer.py then RUN THIS CELL AS IS
!chmod a+x NaiveBayes/train_mapper.py
!chmod a+x NaiveBayes/train_reducer.py
!echo "=========== MAPPER DOCSTRING ============"
!head -n 8 NaiveBayes/train_mapper.py | tail -n 6
!echo "=========== REDUCER DOCSTRING ============"
!head -n 8 NaiveBayes/train_reducer.py | tail -n 6

=========== MAPPER DOCSTRING ============
Mapper reads in text documents and emits word counts by class.
INPUT:                                                    
    DocID \t true_class \t subject \t body                
OUTPUT:                                                   
    partitionKey \t word \t class0_partialCount,class1_partialCount       
    
=========== REDUCER DOCSTRING ============
Reducer aggregates word counts by class and emits frequencies.

INPUT:
    partitionKey \t word \t class0_partialCount,class1_partialCount 
OUTPUT:
    ID \t word \t class0_count,class1count,P(word|class0),P(word|class1)


In [ ]:
### REFERENCE FOR UNIT TEST - REMOVE!!!!!

D1	1		Chinese Beijing Chinese
D2	1		Chinese Chinese Shanghai
D3	1		Chinese Macao
D4	0		Tokyo Japan Chinese


D5	1		Chinese Chinese Chinese Tokyo Japan
D6	1		Beijing Shanghai Trade
D7	0		Japan Macao Tokyo
D8	0		Tokyo Japan Trade

__`part b starts here`:__ MNB _without_ Smoothing (training on Chinese Example Corpus).

In [256]:
# part b - write a unit test for your mapper here
!cat NaiveBayes/chineseTrain.txt | NaiveBayes/train_mapper.py | column -t

A  chinese              0,1
A  beijing              0,1
A  chinese              0,1
A  chinese              0,1
A  chinese              0,1
C  shanghai             0,1
A  chinese              0,1
B  macao                0,1
C  tokyo                1,0
B  japan                1,0
A  chinese              1,0
A  !totalWordCount      3,8
A  !totalDocClassCount  1,3
B  !totalWordCount      3,8
B  !totalDocClassCount  1,3
C  !totalWordCount      3,8
C  !totalDocClassCount  1,3


In [ ]:
### REFERENCE FOR UNIT TEST - REMOVE!!!!!

chinese	1.0,5.0,0.222222222222,0.428571428571
ClassPriors	1.0,3.0,0.25,0.75

beijing	0.0,1.0,0.111111111111,0.142857142857
chinese	1.0,5.0,0.222222222222,0.428571428571
tokyo	1.0,0.0,0.222222222222,0.0714285714286
shanghai	0.0,1.0,0.111111111111,0.142857142857
ClassPriors	1.0,3.0,0.25,0.75
japan	1.0,0.0,0.222222222222,0.0714285714286
macao	0.0,1.0,0.111111111111,0.142857142857

In [284]:
# part b - write a unit test for your reducer here
!cat chineseTemp.txt | NaiveBayes/train_reducer.py | column -t

atest        0.0,  1.0,  0.0,                 0.125
beijing      1.0,  2.0,  0.3333333333333333,  0.25
btest        1.0,  0.0,  0.3333333333333333,  0.0
chinese      2.0,  5.0,  0.6666666666666666,  0.625
ClassPriors  1.0,  3.0,  0.25,                0.75


In [285]:
# part b - write a systems test for your mapper + reducer together here
!cat NaiveBayes/chineseTrain.txt | NaiveBayes/train_mapper.py | sort -k1,1 | NaiveBayes/train_reducer.py | column -t

beijing      0.0,  1.0,  0.0,                 0.125
chinese      1.0,  5.0,  0.3333333333333333,  0.625
japan        1.0,  0.0,  0.3333333333333333,  0.0
macao        0.0,  1.0,  0.0,                 0.125
shanghai     0.0,  1.0,  0.0,                 0.125
tokyo        1.0,  0.0,  0.3333333333333333,  0.0
ClassPriors  1.0,  3.0,  0.25,                0.75


In [ ]:
# part b - clear (and name) an output directory in HDFS for your unsmoothed chinese NB model
!hdfs dfs -rm -r {HDFS_DIR}/chinese-output-nb-unsmoothed

In [292]:
!hdfs dfs -rm -r {HDFS_DIR}/chinese-output-nb-unsmoothed

#   -reducer /bin/cat \ train_reducer.py \

# part b - write your hadoop streaming job
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=3 \
  -D stream.map.output.field.separator="\t" \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k2,2" \
  -D mapreduce.partition.keypartitioner.options="-k1,1" \
  -files NaiveBayes/train_reducer.py,NaiveBayes/train_mapper.py \
  -mapper train_mapper.py \
  -reducer train_reducer.py \
  -input {HDFS_DIR}/chineseTrain.txt \
  -output {HDFS_DIR}/chinese-output-nb-unsmoothed \
  -cmdenv PATH={PATH} \
  -numReduceTasks 3 \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner

Deleted /user/root/HW2/chinese-output-nb-unsmoothed
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob5181737867531070062.jar tmpDir=null
21/01/24 12:31:32 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/24 12:31:32 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/24 12:31:33 INFO mapred.FileInputFormat: Total input paths to process : 1
21/01/24 12:31:33 INFO mapreduce.JobSubmitter: number of splits:2
21/01/24 12:31:33 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1611186257570_0049
21/01/24 12:31:33 INFO impl.YarnClientImpl: Submitted application application_1611186257570_0049
21/01/24 12:31:33 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1611186257570_0049/
21/01/24 12:31:33 INFO mapreduce.Job: Running job: job_1611186257570_0049
21/01/24 12:31:39 INFO mapreduce.Job: Job job_1611186257570_0049 running in uber mode : false
21/01/24 

In [293]:
# part b - extract your results (i.e. model) to a local file
!hdfs dfs -cat {HDFS_DIR}/chinese-output-nb-unsmoothed/part-000* > NaiveBayes/chineseResultsNbUnsmoothed.txt

In [294]:
# part b - print your model so that we can confirm that it matches expected results
!cat NaiveBayes/chineseResultsNbUnsmoothed.txt | column -t

japan        1.0,  0.0,  0.3333333333333333,  0.0
macao        0.0,  1.0,  0.0,                 0.125
ClassPriors  1.0,  3.0,  0.25,                0.75
shanghai     0.0,  1.0,  0.0,                 0.125
tokyo        1.0,  0.0,  0.3333333333333333,  0.0
ClassPriors  1.0,  3.0,  0.25,                0.75
beijing      0.0,  1.0,  0.0,                 0.125
chinese      1.0,  5.0,  0.3333333333333333,  0.625
ClassPriors  1.0,  3.0,  0.25,                0.75


In [296]:
### Added for reference... DELETE before submitting
for p in range(3):
    print('='*10,f'PARTITION{p+1}','='*10)
    !hdfs dfs -cat {HDFS_DIR}/chinese-output-nb-unsmoothed/part-0000{p} | head -n 5

========== PARTITION1 ==========
japan	1.0, 0.0,                  0.3333333333333333, 0.0
macao	0.0, 1.0,      0.0, 0.125
ClassPriors	1.0, 3.0,      0.25, 0.75
========== PARTITION2 ==========
shanghai	0.0, 1.0,                  0.0, 0.125
tokyo	1.0, 0.0,      0.3333333333333333, 0.0
ClassPriors	1.0, 3.0,      0.25, 0.75
========== PARTITION3 ==========
beijing	0.0, 1.0,                  0.0, 0.125
chinese	1.0, 5.0,      0.3333333333333333, 0.625
ClassPriors	1.0, 3.0,      0.25, 0.75


In [ ]:
# Summary of Results - Delete later
# One ClassPrior per Reduce job
beijing      0.0,  1.0,  0.0,                 0.125
chinese      1.0,  5.0,  0.3333333333333333,  0.625
japan        1.0,  0.0,  0.3333333333333333,  0.0
macao        0.0,  1.0,  0.0,                 0.125
shanghai     0.0,  1.0,  0.0,                 0.125
tokyo        1.0,  0.0,  0.3333333333333333,  0.0
ClassPriors  1.0,  3.0,  0.25,                0.75

# Ouput using '-reducer /bin/cat \'
B  !totalDocClassCount  0,2
B  !totalDocClassCount  1,1
B  !totalWordCount      3,2
B  !totalWordCount      0,6
B  japan                1,0
B  macao                0,1
C  !totalDocClassCount  1,1
C  !totalDocClassCount  0,2
C  !totalWordCount      0,6
C  !totalWordCount      3,2
C  shanghai             0,1
C  tokyo                1,0
A  !totalDocClassCount  1,1
A  !totalDocClassCount  0,2
A  !totalWordCount      0,6
A  !totalWordCount      3,2
A  beijing              0,1
A  chinese              0,1
A  chinese              0,1
A  chinese              0,1
A  chinese              0,1
A  chinese              1,0
A  chinese              0,1

In [295]:
!cat NaiveBayes/chineseTrain.txt

D1	1		Chinese Beijing Chinese
D2	1		Chinese Chinese Shanghai
D3	1		Chinese Macao
D4	0		Tokyo Japan Chinese


__`part e starts here`:__ MNB _with_ Smoothing (training on Chinese Example Corpus).

In [311]:
# part e - write a unit test for your NEW reducer here
!chmod a+x NaiveBayes/train_reducer_smooth.py
!cat chineseTemp.txt | NaiveBayes/train_reducer_smooth.py | column -t

atest        0.0,  1.0,  0.0,                 0.125
beijing      1.0,  2.0,  0.3333333333333333,  0.25
btest        1.0,  0.0,  0.3333333333333333,  0.0
chinese      2.0,  5.0,  0.6666666666666666,  0.625
ClassPriors  1.0,  3.0,  0.25,                0.75
!TotalTerms  4.0,  4.0,  1.0,                 1.0


In [343]:
# part e - write a systems test for your mapper + reducer together here
!cat NaiveBayes/chineseTrain.txt | NaiveBayes/train_mapper.py | sort -k1,1 | NaiveBayes/train_reducer_smooth.py | column -t

beijing      0.0,  1.0,  0.0,                 0.125
chinese      1.0,  5.0,  0.3333333333333333,  0.625
japan        1.0,  0.0,  0.3333333333333333,  0.0
macao        0.0,  1.0,  0.0,                 0.125
shanghai     0.0,  1.0,  0.0,                 0.125
tokyo        1.0,  0.0,  0.3333333333333333,  0.0
ClassPriors  1.0,  3.0,  0.25,                0.75
!TotalTerms  6.0,  6.0,  3.0,                 8.0


In [344]:
# part e - clear (and name) an output directory in HDFS for your SMOOTHED chinese NB model
!hdfs dfs -rm -r {HDFS_DIR}/chinese-output-nb-smoothed

Deleted /user/root/HW2/chinese-output-nb-smoothed


In [345]:
!hdfs dfs -rm -r {HDFS_DIR}/chinese-output-nb-smoothed

# part e - write your hadoop streaming job
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=3 \
  -D stream.map.output.field.separator="\t" \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k2,2" \
  -D mapreduce.partition.keypartitioner.options="-k1,1" \
  -files NaiveBayes/train_reducer_smooth.py,NaiveBayes/train_mapper.py \
  -mapper train_mapper.py \
  -reducer train_reducer_smooth.py \
  -input {HDFS_DIR}/chineseTrain.txt \
  -output {HDFS_DIR}/chinese-output-nb-smoothed \
  -cmdenv PATH={PATH} \
  -numReduceTasks 3 \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner

rm: `/user/root/HW2/chinese-output-nb-smoothed': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob48120625123920153.jar tmpDir=null
21/01/24 15:52:46 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/24 15:52:46 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/24 15:52:46 INFO mapred.FileInputFormat: Total input paths to process : 1
21/01/24 15:52:47 INFO mapreduce.JobSubmitter: number of splits:2
21/01/24 15:52:47 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1611186257570_0053
21/01/24 15:52:47 INFO impl.YarnClientImpl: Submitted application application_1611186257570_0053
21/01/24 15:52:47 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1611186257570_0053/
21/01/24 15:52:47 INFO mapreduce.Job: Running job: job_1611186257570_0053
21/01/24 15:52:53 INFO mapreduce.Job: Job job_1611186257570_0053 running in uber m

In [346]:
# part e - extract your results (i.e. model) to a local file
!hdfs dfs -cat {HDFS_DIR}/chinese-output-nb-smoothed/part-000* > NaiveBayes/chineseResultsNbSmoothed.txt

In [347]:
# part e - print your model (above local file) so that we can confirm that it matches expected results
!cat NaiveBayes/chineseResultsNbSmoothed.txt | column -t

japan        1.0,  0.0,  0.3333333333333333,  0.0
macao        0.0,  1.0,  0.0,                 0.125
ClassPriors  1.0,  3.0,  0.25,                0.75
!TotalTerms  2.0,  2.0,  1.0,                 1.0
shanghai     0.0,  1.0,  0.0,                 0.125
tokyo        1.0,  0.0,  0.3333333333333333,  0.0
ClassPriors  1.0,  3.0,  0.25,                0.75
!TotalTerms  2.0,  2.0,  1.0,                 1.0
beijing      0.0,  1.0,  0.0,                 0.125
chinese      1.0,  5.0,  0.3333333333333333,  0.625
ClassPriors  1.0,  3.0,  0.25,                0.75
!TotalTerms  2.0,  2.0,  1.0,                 6.0


In [ ]:
# part e - post processing
# Post Processing to get 'B' term for Smoothed Model

In [375]:
# Unit test new reducer for post processing
!chmod a+x NaiveBayes/train_reducer_smooth_post.py
!cat chineseTempSorted.txt | NaiveBayes/train_reducer_smooth_post.py | column -t

beijing      0.0,      1.0,       0.1111111111111111,  0.14285714285714285
chinese      1.0,      5.0,       0.2222222222222222,  0.42857142857142855
ClassPriors  1.0,3.0,  0.25,0.75
japan        1.0,      0.0,       0.2222222222222222,  0.07142857142857142
macao        0.0,      1.0,       0.1111111111111111,  0.14285714285714285
shanghai     0.0,      1.0,       0.1111111111111111,  0.14285714285714285
tokyo        1.0,      0.0,       0.2222222222222222,  0.07142857142857142


In [ ]:
### For Reference only

beijing	0.0,1.0,0.111111111111,0.142857142857
chinese	1.0,5.0,0.222222222222,0.428571428571
tokyo	1.0,0.0,0.222222222222,0.0714285714286
shanghai	0.0,1.0,0.111111111111,0.142857142857
ClassPriors	1.0,3.0,0.25,0.75
japan	1.0,0.0,0.222222222222,0.0714285714286
macao	0.0,1.0,0.111111111111,0.142857142857

In [ ]:
# part e - write a systems test for your mapper + reducer together here
!cat NaiveBayes/chineseTrain.txt | NaiveBayes/train_mapper.py | sort -k1,1 | NaiveBayes/train_reducer_smooth.py | column -t

# Question 9: Enron Ham/Spam NB Classifier & Results.

Fantastic work. We're finally ready to perform Spam Classification on the Enron Corpus. In this question you'll run the analysis you've developed, report its performance.

### Q9 Tasks:
* __a) train/test split:__ Run the provided code to split our Enron file into a training set and testing set then load them into HDFS. [`NOTE:` _Make sure you re calculate the vocab size for just the training set!_]

* __b) train 2 models:__ Write Hadoop Streaming jobs to train MNB Models on the training set with and without smoothing. Save your models to local files at __`NaiveBayes/Unsmoothed/NBmodel.txt`__ and __`NaiveBayes/Smoothed/NBmodel.txt`__. [`NOTE:` _This naming is important because we wrote our classification task so that it expects a file of that name... if this inelegance frustrates you there is an alternative that would involve a few adjustments to your code [read more about it here](http://www.tnoda.com/blog/2013-11-23)._] Finally run the checks that we provide to confirm that your results are correct.


* __c) code:__ Recall that we designed our classification job with just a mapper. An efficient way to report the performance of our models would be to simply add a reducer phase to this job and compute precision and recall right there. Complete the code in __`NaiveBayes/evaluation_reducer.py`__ and then write Hadoop jobs to evaluate your two models on the test set. Report their performance side by side. [`NOTE:` if you need a refresher on precision, recall and F1-score [Wikipedia](https://en.wikipedia.org/wiki/F1_score) is a good resource.]


* __d) short response:__ Compare the performance of your two models. What do you notice about the unsmoothed model's predictions? Can you guess why this is happening? Which evaluation measure do you think is most relevant in our use case? [`NOTE:` _Feel free to answer using your common sense but if you want more information on evaluating the classification task checkout_ [this blogpost](https://tryolabs.com/blog/2013/03/25/why-accuracy-alone-bad-measure-classification-tasks-and-what-we-can-do-about-it/
) or [this paper](http://www.flinders.edu.au/science_engineering/fms/School-CSEM/publications/tech_reps-research_artfcts/TRRA_2007.pdf
)]



### Q9 Student Answers:
> __d)__ Type your answer here!



__Test/Train split__

In [234]:
# part a - test/train split (RUN THIS CELL AS IS)
!head -n 80 data/enronemail_1h.txt > data/enron_train.txt
!tail -n 20 data/enronemail_1h.txt > data/enron_test.txt
!hdfs dfs -copyFromLocal data/enron_train.txt {HDFS_DIR}
!hdfs dfs -copyFromLocal data/enron_test.txt {HDFS_DIR}

__Training__ (Enron MNB Model _without smoothing_ )

In [ ]:
# part b -  Unsmoothed model (FILL IN THE MISSING CODE BELOW)

# clear the output directory
!hdfs dfs -rm -r {HDFS_DIR}/enron-model

# hadoop command
!hadoop jar {JAR_FILE} \








# save the model locally
!mkdir NaiveBayes/Unsmoothed
!hdfs dfs -cat {HDFS_DIR}/enron-model/part-000* > NaiveBayes/Unsmoothed/NBmodel.txt

In [ ]:
# part b - check your UNSMOOTHED model results (RUN THIS CELL AS IS)
!grep assistance NaiveBayes/Unsmoothed/NBmodel.txt
# EXPECTED OUTPUT: assistance	2,4,0.000172547666293,0.000296823983378

In [ ]:
# part b - check your UNSMOOTHED model results (RUN THIS CELL AS IS)
!grep money NaiveBayes/Unsmoothed/NBmodel.txt
# EXPECTED OUTPUT: money	1,22,8.62738331464e-05,0.00163253190858

__Training__ (Enron MNB Model _with Laplace +1 smoothing_ )

In [ ]:
# part b -  Smoothed model (FILL IN THE MISSING CODE BELOW)

# clear the output directory
!hdfs dfs -rm -r {HDFS_DIR}/smooth-model

# hadoop command
!hadoop jar {JAR_FILE} \







# save the model locally
!mkdir NaiveBayes/Unsmoothed
!hdfs dfs -cat {HDFS_DIR}/smooth-model/part-000* > NaiveBayes/Smoothed/NBmodel.txt

In [ ]:
# part b - check your SMOOTHED model results (RUN THIS CELL AS IS)
!grep assistance NaiveBayes/Smoothed/NBmodel.txt
# EXPECTED OUTPUT: assistance	2,4,0.000185804533631,0.000277300205202

In [ ]:
# part b - check your SMOOTHED model results (RUN THIS CELL AS IS)
!grep money NaiveBayes/Smoothed/NBmodel.txt
# EXPECTED OUTPUT: money	1,22,0.000123869689087,0.00127558094393

__Evaluation__

In [ ]:
# part c - write your code in NaiveBayes/evaluation_reducer.py then RUN THIS
!chmod a+x NaiveBayes/evaluation_reducer.py

In [ ]:
# part c - unit test your evaluation job on the chinese model (RUN THIS CELL AS IS)
!cat NaiveBayes/chineseTest.txt | NaiveBayes/classify_mapper.py 
!cat NaiveBayes/chineseTest.txt | NaiveBayes/classify_mapper.py | NaiveBayes/evaluation_reducer.py

In [ ]:
# part c - Evaluate the UNSMOOTHED Model Here (FILL IN THE MISSING CODE)

# clear output directory

# hadoop job





# retrieve results locally

In [ ]:
# part c - Evaluate the SMOOTHED Model Here (FILL IN THE MISSING CODE)

# clear output directory

# hadoop job





# retrieve results locally

In [ ]:
# part c - display results 
# NOTE: feel free to modify the tail commands to match the format of your results file
print('=========== UNSMOOTHED MODEL ============')
!tail -n 9 NaiveBayes/Unsmoothed/results.txt
print('=========== SMOOTHED MODEL ============')
!tail -n 9 NaiveBayes/Smoothed/results.txt

__`EXPECTED RESULTS:`__ 
<table>
<th>Unsmoothed Model</th>
<th>Smoothed Model</th>
<tr>
<td><pre>
# Documents:	20
True Positives:	1
True Negatives:	9
False Positives:	0
False Negatives:	10
Accuracy	0.5
Precision	1.0
Recall	0.0909
F-Score	0.1666
</pre></td>
<td><pre>
# Documents:	20
True Positives:	11
True Negatives:	6
False Positives:	3
False Negatives:	0
Accuracy	0.85
Precision	0.7857
Recall	1.0
F-Score	0.88
</pre></td>
</tr>
</table>

__`NOTE:`__ _Don't be too disappointed if these seem low to you. We've trained and tested on a very very small corpus... bigger datasets coming soon!_

# Question 10: Custom Partitioning and Secondary Sort

Now that we have our model, we can analyse the results and think about future improvements.

### Q10 Tasks:

* __a) code + short response:__ Let's look at the top ten words with the highest conditional probability in `Spam` and in `Ham`. We'll do this by writing a Hadoop job that sorts the model file (`NaiveBayes/Smoothed/NBmodel.py`). Normally we'd have to run two jobs -- one that sorts on $P(word|ham)$ and another that sorts on $P(word|spam)$. However if we slighly modify the data format in the model file then we can get the top words in each class with just one job. We've written a mapper that will do just this for you. Read through __`NaiveBayes/model_sort_mapper.py`__ and then briefly explain how this mapper will allow us to partition and sort our model file. Write a Hadoop job that uses our mapper and `/bin/cat` for a reducer to partition and sort. Print out the top 10 words in each class (where 'top' == highest conditional probability).[`HINT:` _this should remind you a lot of what we did in Question 6._]


* __b) short response:__ What do you notice about the 'top words' we printed in `a`? How would increasing the smoothing parameter 'k' affect the probabilities for the top words that you identified for 'a'. How would they affect the probabilities of words that occur much more in one class than another? In summary, how does the smoothing parameter 'k' affect the bias and the variance of our model. [`NOTE:` _you do not need to code anything for this task, but if you are struggling with it you could try changing 'k' and see what happens to the test set. We don't recommend doing this exploration with the Enron data because it will be harder to see the impact with such a big vocabulary_]

### Q10 Student Answers:
> __a)__ Type your answer here!

> __b)__ Type your answer here!

In [ ]:
# part a - write your Hadoop job here (sort smoothed model on P(word|class))

# clear output directory

# hadoop job

In [ ]:
# part b - print top words in each class


### Congratulations, you have completed HW2! Please refer to the readme for submission instructions.

If you would like to provide feedback regarding this homework, please use the survey at: https://docs.google.com/forms/d/e/1FAIpQLSce9feiQeSkdP43A0ZYui1tMGIBfLfzb0rmgToQeZD9bXXX8Q/viewform